In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools as iter
import sqlite3

In [2]:
df1 = pd.DataFrame(np.random.rand(3,3) ,columns=["p_12","p_22","p_12"])
_test = (2,3)
for _y in range(1, _test[1]+1):
    for _x in range(1, _test[0]+1):
        print("p_{y}{x}".format(x=_x,y=_y))

insert_data = pd.Series([1,2,3], index=df1.columns, name="test")
df1 = df1.append(insert_data)
df1

p_11
p_12
p_21
p_22
p_31
p_32


,p_12,p_22,p_12
0,0.793937,0.985155,0.580987
1,0.501882,0.501192,0.016858
2,0.783639,0.040453,0.515278
test,1.000000,2.000000,3.000000


In [3]:
def generatePMatrix():
  _temp = np.zeros((2 ** 16,4,4))
  _count = 0
  for p_11, p_12, p_13, p_14, p_21, p_22, p_23, p_24, p_31, p_32, p_33, p_34, p_41, p_42, p_43, p_44 in iter.product([0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1]):
    _temp[_count] = np.array([[p_11, p_12, p_13, p_14],[p_21, p_22, p_23, p_24],[p_31, p_32, p_33, p_34],[p_41, p_42, p_43, p_44]]) 
    _count += 1
  
  return _temp

p_matrix = generatePMatrix()
p_matrix.shape

(65536, 4, 4)

In [5]:
def calcQMatrix(data = p_matrix, p_size=(4,4),a_size=(2,2)):
    q_size = (p_size[0]-a_size[0]+1, p_size[1]-a_size[1]+1)
    _columns = []
    
    for _y in range(1, p_size[1]+1):
        for _x in range(1, p_size[0]+1):
            _index = "p_{y}{x}".format(x = _x, y = _y)
            _columns.append(_index)
    
    for _y in range(1, q_size[1]+1):
        for _x in range(1, q_size[0]+1):
            _index = "q_{y}{x}".format(x = _x, y = _y)
            _columns.append(_index)
            
    
    df = pd.DataFrame(columns=_columns)
    _name_count = 0
    
    for i in range( data.shape[0] ):
        _data = data[i]
        for a_11, a_12, a_21, a_22 in iter.product([0,1],[0,1],[0,1],[0,1]):
            q_11 = _data[0][0] * a_11 + _data[0][1] * a_12 + _data[1][0] * a_21 + _data[1][1] * a_22
            q_12 = _data[0][1] * a_11 + _data[0][2] * a_12 + _data[1][1] * a_21 + _data[1][2] * a_22
            q_13 = _data[0][2] * a_11 + _data[0][3] * a_12 + _data[1][2] * a_21 + _data[1][3] * a_22
            
            q_21 = _data[1][0] * a_11 + _data[1][1] * a_12 + _data[2][0] * a_21 + _data[2][1] * a_22
            q_22 = _data[1][1] * a_11 + _data[1][2] * a_12 + _data[2][1] * a_21 + _data[2][2] * a_22
            q_23 = _data[1][2] * a_11 + _data[1][3] * a_12 + _data[2][2] * a_21 + _data[2][3] * a_22
            
            q_31 = _data[2][0] * a_11 + _data[2][1] * a_12 + _data[3][0] * a_21 + _data[3][1] * a_22
            q_32 = _data[2][1] * a_11 + _data[2][2] * a_12 + _data[3][1] * a_21 + _data[3][2] * a_22
            q_33 = _data[2][2] * a_11 + _data[2][3] * a_12 + _data[3][2] * a_21 + _data[3][3] * a_22
            
            _insert = pd.Series([_data[0][0], _data[0][1], _data[0][2], _data[0][3],
                                 _data[1][0], _data[1][1], _data[1][2], _data[1][3],
                                 _data[2][0], _data[2][1], _data[2][2], _data[2][3],
                                 _data[3][0], _data[3][1], _data[3][2], _data[3][3],
                                 q_11, q_12, q_13,
                                 q_21, q_22, q_23,
                                 q_31, q_32, q_33], index=df.columns, name = _name_count)
            df = df.append(_insert)
            _name_count += 1
    
    df.head(5)
    df.tail(5)
    return df
df = calcQMatrix()

KeyboardInterrupt: 

In [3]:
df = pd.read_csv("../4x4matrix.csv")

In [5]:
df.head()
df.tail()

,Unnamed: 0,p_11,p_12,p_13,p_14,p_21,p_22,p_23,p_24,p_31,...,p_44,q_11,q_12,q_13,q_21,q_22,q_23,q_31,q_32,q_33
1048571,1048571,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
1048572,1048572,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1048573,1048573,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
1048574,1048574,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
1048575,1048575,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0


In [21]:
temp = df.iloc[0,17:]
np.array(temp)

array([0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [45]:
df.query('p_11 == 0 and p_12 == 0 and p_13 == 0')

,Unnamed: 0,p_11,p_12,p_13,p_14,p_21,p_22,p_23,p_24,p_31,...,p_44,q_11,q_12,q_13,q_21,q_22,q_23,q_31,q_32,q_33
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,Unnamed: 0,p_11,p_12,p_13,p_14,p_21,p_22,p_23,p_24,p_31,...,p_44,q_11,q_12,q_13,q_21,q_22,q_23,q_31,q_32,q_33
count,1.048576e+06,1048576.0,1048576.0,1048576.0,1048576.0,1048576.0,1048576.0,1048576.0,1048576.0,1048576.0,...,1048576.0,1.048576e+06,1.048576e+06,1.048576e+06,1.048576e+06,1.048576e+06,1.048576e+06,1.048576e+06,1.048576e+06,1.048576e+06
mean,5.242875e+05,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
std,3.026980e+05,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,8.660258e-01,8.660258e-01,8.660258e-01,8.660258e-01,8.660258e-01,8.660258e-01,8.660258e-01,8.660258e-01,8.660258e-01
min,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.621438e+05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,5.242875e+05,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
75%,7.864312e+05,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00
max,1.048575e+06,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00,4.000000e+00
